# Transformation of Screen Scrape Data to Clean CSV

## ETL: Read File, Clean File, Save File

Import the data provided in the `flight_info.csv` file from the source website. Then, clean up the file and output it as a new CSV


In [1]:
# Import dependencies
from pprint import pprint
import pandas as pd
import datetime

In [2]:
# Import our source data for cleansing...
df_flights = pd.read_csv("./old_files/flight_info.csv")
df_flights.head()

,Plane ID,Flight Date,Departure Airport,Arrival Airport,Flight Distance,Flight Time,Fuel Used
0,N1980K,NaN,NaN,NaN,NaN,NaN,NaN
1,N1980K,07/09/2023,lax,bri,NaN,12 hours17 minutes,NaN
2,N1980K,07/31/2023,lax,nap,NaN,12 hours8 minutes,"6,100 gallons"
3,N1980K,02/24/2023,cma,lin,NaN,11 hours24 minutes,170 gallons
4,N1980K,08/06/2023,goa,qlt,NaN,11 hours18 minutes,"5,700 gallons"


In [3]:
# Delete rows with an NaN for Flight Date as they are not valid records--remainder from the table structure on the website.
df_flights = df_flights.dropna(subset=['Flight Date'])
df_flights = df_flights.dropna(subset=['Arrival Airport'])
df_flights = df_flights.dropna(subset=['Fuel Used'])


In [4]:
# We can also drop some columns we don't need...
df_flights = df_flights.drop(['Flight Distance', 'Flight Time'], axis = 1)

In [5]:
# And we need to rename some columns to match the expected format...
df_flights = df_flights.rename(columns={'Plane ID': 'tail_number', 
                        'Flight Date': 'dep_date_time', 
                        'Departure Airport': 'dep_airport',
                        'Arrival Airport': 'arr_airport',
                        'Fuel Used': 'fuel_gallons'})

df_flights.head()

,tail_number,dep_date_time,dep_airport,arr_airport,fuel_gallons
2,N1980K,07/31/2023,lax,nap,"6,100 gallons"
3,N1980K,02/24/2023,cma,lin,170 gallons
4,N1980K,08/06/2023,goa,qlt,"5,700 gallons"
5,N1980K,07/26/2023,kix,san,"5,600 gallons"
6,N1980K,06/07/2023,sxf,cma,"5,600 gallons"


In [6]:
# Now we need to force the airport codes to be upper case...
df_flights['dep_airport'] = df_flights['dep_airport'].str.upper()
df_flights['arr_airport'] = df_flights['arr_airport'].str.upper()

df_flights.head()

,tail_number,dep_date_time,dep_airport,arr_airport,fuel_gallons
2,N1980K,07/31/2023,LAX,NAP,"6,100 gallons"
3,N1980K,02/24/2023,CMA,LIN,170 gallons
4,N1980K,08/06/2023,GOA,QLT,"5,700 gallons"
5,N1980K,07/26/2023,KIX,SAN,"5,600 gallons"
6,N1980K,06/07/2023,SXF,CMA,"5,600 gallons"


In [7]:
# Now we need to get rid of the trailing ' gallons' from all our fuel_gallons values
# so we will be able to convert them to numbers...

df_flights['fuel_gallons'] = df_flights['fuel_gallons'].str[:-8]
df_flights.head()

,tail_number,dep_date_time,dep_airport,arr_airport,fuel_gallons
2,N1980K,07/31/2023,LAX,NAP,"6,100"
3,N1980K,02/24/2023,CMA,LIN,170
4,N1980K,08/06/2023,GOA,QLT,"5,700"
5,N1980K,07/26/2023,KIX,SAN,"5,600"
6,N1980K,06/07/2023,SXF,CMA,"5,600"


In [8]:
df_flights['fuel_gallons'] = df_flights['fuel_gallons'].str.replace(',', '')
df_flights.head()

,tail_number,dep_date_time,dep_airport,arr_airport,fuel_gallons
2,N1980K,07/31/2023,LAX,NAP,6100
3,N1980K,02/24/2023,CMA,LIN,170
4,N1980K,08/06/2023,GOA,QLT,5700
5,N1980K,07/26/2023,KIX,SAN,5600
6,N1980K,06/07/2023,SXF,CMA,5600


In [9]:
df_flights['fuel_gallons'] = df_flights['fuel_gallons'].astype(int)
df_flights.head()

,tail_number,dep_date_time,dep_airport,arr_airport,fuel_gallons
2,N1980K,07/31/2023,LAX,NAP,6100
3,N1980K,02/24/2023,CMA,LIN,170
4,N1980K,08/06/2023,GOA,QLT,5700
5,N1980K,07/26/2023,KIX,SAN,5600
6,N1980K,06/07/2023,SXF,CMA,5600


In [10]:
# Note that our dep_date_time is still a text string, but I don't think we care
# since it is just going to be stored as a CSV without a datatype, and then imported
# into a SQL database that has a defined schema that can read the source data
# just fine.

df_flights['dep_date_time'] = pd.to_datetime(df_flights['dep_date_time'])
print(df_flights.dtypes)

tail_number              object
dep_date_time    datetime64[ns]
dep_airport              object
arr_airport              object
fuel_gallons              int64
dtype: object


In [11]:
# Last bit of cleanup is to reorder our columns to match what our SQL schema is expecting...

df_flights = df_flights[['dep_airport', 'arr_airport', 'dep_date_time', 'fuel_gallons', 'tail_number']]
df_flights.head()

,dep_airport,arr_airport,dep_date_time,fuel_gallons,tail_number
2,LAX,NAP,2023-07-31,6100,N1980K
3,CMA,LIN,2023-02-24,170,N1980K
4,GOA,QLT,2023-08-06,5700,N1980K
5,KIX,SAN,2023-07-26,5600,N1980K
6,SXF,CMA,2023-06-07,5600,N1980K


In [12]:
# Looks like we have some duplicates that need removal... Again, a legacy of how the tables were
# managed in the source website.

df_flights = df_flights.drop_duplicates()

In [13]:
# We also want to look at only a certain range of dates, from 5/29/23 to 5/29/24...

cutoff_date = pd.to_datetime('2023-05-29')

# Drop rows where the 'dep_date_time' is before the cutoff date
df_flights = df_flights[df_flights['dep_date_time'] >= cutoff_date]

df_flights.head()

,dep_airport,arr_airport,dep_date_time,fuel_gallons,tail_number
2,LAX,NAP,2023-07-31,6100,N1980K
4,GOA,QLT,2023-08-06,5700,N1980K
5,KIX,SAN,2023-07-26,5600,N1980K
6,SXF,CMA,2023-06-07,5600,N1980K
7,CMA,SXF,2023-06-06,5600,N1980K


In [14]:
# Now we can write our dataframe to CSV

df_flights.to_csv('./static/data/flights.csv', index=False)